In [16]:
import os
import numpy as np
from openai import OpenAI
import gradio as gr

from google.colab import files

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document

import faiss
import json
from pypdf import PdfReader

# ⚠️ Mets ta clé ici UNE FOIS (ou utilise une variable d'environnement Colab)
os.environ["OPENAI_API_KEY"] = os.environ.get("OPENAI_API_KEY", "TA CLE API")

client = OpenAI(api_key=os.environ["OPENAI_API_KEY"])


In [17]:
def lire_txt(path: str) -> str:
    with open(path, "r", encoding="utf-8", errors="ignore") as f:
        return f.read()

def lire_pdf(path: str) -> str:
    texte = ""
    with open(path, "rb") as f:
        reader = PdfReader(f)
        for page in reader.pages:
            page_text = page.extract_text() or ""
            texte += page_text + "\n"
    return texte

def charger_documents_depuis_fichiers(filenames):
    docs = []
    for filename in filenames:
        if filename.lower().endswith(".txt"):
            content = lire_txt(filename)
        elif filename.lower().endswith(".pdf"):
            content = lire_pdf(filename)
        else:
            print(f"Format non supporté : {filename}, ignoré.")
            continue

        # On crée un Document LangChain pour garder les métadonnées
        docs.append(
            Document(
                page_content=content,
                metadata={"source": filename}
            )
        )
    return docs

# Upload de fichiers depuis ton PC
uploaded = files.upload()  # choisis tes .txt / .pdf
filenames = list(uploaded.keys())

documents = charger_documents_depuis_fichiers(filenames)

print(f"{len(documents)} documents chargés :")
for d in documents:
    print("-", d.metadata["source"], "(", len(d.page_content), "caractères )")


Saving procedure_exemple.pdf to procedure_exemple (1).pdf
1 documents chargés :
- procedure_exemple (1).pdf ( 510 caractères )


In [18]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=800,
    chunk_overlap=200,
    length_function=len,
    separators=["\n\n", "\n", ". ", " ", ""]
)

chunks = text_splitter.split_documents(documents)

print("Nombre total de chunks :", len(chunks))
print("Exemple de chunk :\n")
print(chunks[0].page_content[:500], "...")
print("Source du chunk :", chunks[0].metadata["source"])


Nombre total de chunks : 1
Exemple de chunk :

Procédure d’Accès aux Outils Internes
-------------------------------------
1. Objectif :
Cette procédure décrit les étapes permettant à un collaborateur d’obtenir un accès aux outils
digitaux internes.
2. Étapes :
- Le collaborateur soumet une demande via le portail interne.
- Le manager valide la demande.
- L’équipe IT reçoit la demande et créé le compte associé.
- Le collaborateur reçoit un email de confirmation avec ses identifiants.
3. Support :
En cas de problème, contacter support@entrepr ...
Source du chunk : procedure_exemple (1).pdf


In [19]:
EMBEDDINGS_PATH = "faiss_index.bin"
CHUNKS_PATH = "chunks_meta.json"

def embed_texts(texts):
    response = client.embeddings.create(
        model="text-embedding-3-small",
        input=texts
    )
    return np.array([item.embedding for item in response.data], dtype="float32")

def construire_ou_charger_index(chunks):
    if os.path.exists(EMBEDDINGS_PATH) and os.path.exists(CHUNKS_PATH):
        print("➡️ Chargement de l'index FAISS et des métadonnées depuis le cache...")
        index = faiss.read_index(EMBEDDINGS_PATH)
        with open(CHUNKS_PATH, "r", encoding="utf-8") as f:
            chunks_meta = json.load(f)
        return index, chunks_meta

    print("➡️ Aucun index existant, construction en cours...")

    texts = [c.page_content for c in chunks]
    embeddings = embed_texts(texts)

    dim = embeddings.shape[1]
    index = faiss.IndexFlatIP(dim)  # produit scalaire (pour similarité cosinus approx)
    faiss.normalize_L2(embeddings)
    index.add(embeddings)

    faiss.write_index(index, EMBEDDINGS_PATH)

    chunks_meta = []
    for c in chunks:
        chunks_meta.append({
            "content": c.page_content,
            "source": c.metadata.get("source", "inconnu")
        })

    with open(CHUNKS_PATH, "w", encoding="utf-8") as f:
        json.dump(chunks_meta, f, ensure_ascii=False, indent=2)

    return index, chunks_meta

index, chunks_meta = construire_ou_charger_index(chunks)
print("Index dimension :", index.d)
print("Nombre de vecteurs indexés :", index.ntotal)


➡️ Aucun index existant, construction en cours...
Index dimension : 1536
Nombre de vecteurs indexés : 1


In [20]:
def rechercher_chunks(question: str, top_k: int = 8):
    # Embedding de la question
    q_emb = embed_texts([question])
    faiss.normalize_L2(q_emb)

    # Recherche FAISS
    distances, indices = index.search(q_emb, top_k)
    indices = indices[0]
    distances = distances[0]

    candidats = []
    for idx, dist in zip(indices, distances):
        if idx == -1:
            continue
        meta = chunks_meta[int(idx)]
        candidats.append({
            "score_vec": float(dist),
            "source": meta["source"],
            "content": meta["content"]
        })

    return candidats

def reranker_llm(question: str, candidats, top_n: int = 3):
    """
    Reranker simple : on donne au LLM les candidats, il nous dit lesquels sont les plus pertinents.
    Version simple pour comprendre le concept.
    """
    if len(candidats) <= top_n:
        return candidats

    # On construit un prompt avec les passages numérotés
    passages_text = ""
    for i, c in enumerate(candidats):
        passages_text += f"[{i}] (source: {c['source']})\n{c['content']}\n\n"

    prompt = (
        "Tu es un système d'aide à la recherche d'information.\n"
        "On te donne une question et plusieurs passages de texte.\n"
        "Ta tâche est de sélectionner les passages les plus pertinents pour répondre à la question.\n\n"
        f"QUESTION : {question}\n\n"
        f"PASSAGES CANDIDATS :\n{passages_text}\n"
        "Donne uniquement une liste des indices des 3 meilleurs passages, au format : 0,2,5\n"
    )

    resp = client.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {"role": "system", "content": "Tu es un assistant qui rerank des passages."},
            {"role": "user", "content": prompt}
        ]
    )

    answer = resp.choices[0].message.content
    # On essaie de parser des nombres
    indices_str = [s.strip() for s in answer.replace("\n", "").split(",") if s.strip().isdigit()]
    indices_sel = []
    for s in indices_str:
        i = int(s)
        if 0 <= i < len(candidats):
            indices_sel.append(i)

    if not indices_sel:  # fallback si le parsing foire
        indices_sel = list(range(min(top_n, len(candidats))))

    reranked = [candidats[i] for i in indices_sel[:top_n]]
    return reranked


In [21]:
def repondre_question_rag(question: str, top_k: int = 8, top_n: int = 3):
    candidats = rechercher_chunks(question, top_k=top_k)
    meilleurs = reranker_llm(question, candidats, top_n=top_n)

    if not meilleurs:
        return "Je n'ai trouvé aucun passage pertinent dans les documents.", []

    contexte = ""
    for m in meilleurs:
        contexte += f"[Source: {m['source']}]\n{m['content']}\n\n"

    prompt_user = (
        "Tu es un assistant interne d'entreprise.\n"
        "Tu disposes de documents internes dans le CONTEXTE ci-dessous.\n"
        "Tu dois répondre à la QUESTION en te basant UNIQUEMENT sur ces documents.\n\n"
        "Règles importantes :\n"
        "- Si l'information n'est pas présente dans le contexte, dis clairement que tu ne sais pas.\n"
        "- Ne fais pas de suppositions, ne propose pas de procédures inventées.\n"
        "- Réponds en français, de manière claire, structurée et professionnelle.\n\n"
        f"CONTEXTE :\n{contexte}\n"
        f"QUESTION : {question}\n\n"
        "Réponse :"
    )

    resp = client.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {"role": "system", "content": "Assistant interne fiable, qui ne sort pas du contexte."},
            {"role": "user", "content": prompt_user}
        ]
    )

    answer = resp.choices[0].message.content
    return answer, meilleurs


In [22]:
def repondre_chat(message, history):
    if not message or message.strip() == "":
        return "Merci de poser une question sur vos documents internes. 🙂"

    answer, sources = repondre_question_rag(message, top_k=8, top_n=3)

    sources_text = "\n".join(
        [f"- {s['source']}" for s in sources]
    )

    answer_with_sources = answer + "\n\n---\nSources utilisées :\n" + sources_text
    return answer_with_sources

demo = gr.ChatInterface(
    fn=repondre_chat,
    title="🤖 Chatbot RAG avancé (docs internes)",
    description=(
        "Ce chatbot répond aux questions en se basant sur les documents PDF/TXT fournis.\n"
        "Il utilise un pipeline RAG avec index vectoriel, reranking et guardrails anti-hallucination."
    )
)

demo.launch()


/usr/local/lib/python3.12/dist-packages/gradio/chat_interface.py:347: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://2ceb0f0670c2cf09f2.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
